In [1]:
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "udemy-gstt-to-translate-test-4d758b7eaf66.json"

In [3]:
#!pip install -U protobuf

#!pip3 install --force-reinstall google-cloud-speech
#!pip uninstall protobuf
#!pip install --no-binary protobuf protobuf

#!pip install --upgrade google-cloud-translate


Translation API サンプルコード
https://cloud.google.com/translate/docs/advanced/translate-text-advance?hl=ja#translate_text_example
STT APIサンプルコード
https://cloud.google.com/speech-to-text/docs/streaming-recognize?hl=ja#performing_streaming_speech_recognition_on_an_audio_stream
Google Cloud ダッシュボードコンソール・プロジェクト
https://console.cloud.google.com/apis/dashboard?hl=ja&project=udemy-gstt-test2w

PCマイクに向かって一語づつ区切って、間を空けてしゃべった単語
学校
病院
市役所
駐車場
家電製品
台所
家賃
玄関
住宅

In [4]:
from __future__ import division

import re
import sys

#from google.cloud import speech
from google.cloud import speech_v1 as speech



import pyaudio
from six.moves import queue
from google.cloud import translate

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)

# Imports the Google Cloud Translation library
# Initialize Translation client
def translate_text(text, project_id="udemy-gstt-to-translate-test"):
    """Translating Text."""

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "source_language_code": "ja",
            "target_language_code": "en",
        }
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        print("Translated text: {}".format(translation.translated_text))


def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if not response.results and not response.results[0].alternatives:
            continue

        transcript = response.results[0].alternatives[0].transcript
        
        #if result.is_final:
        if response.results[0].is_final:
            print(transcript)
            translate_text(text=transcript)
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                break
    #for response in responses:
        #if not response.results or not result.alternatives:
            #continue

     
        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        #result = response.results[0]
        #if not result.alternatives:
            #continue

        # Display the transcription of the top alternative.
        #transcript = result.alternatives[0].transcript
        #if result.is_final:
            #print(transcript)
            #translate_text(text=transcript)
            #if re.search(r"\b(exit|quit)\b", transcript, re.I):
                #print("Exiting..")
                #break

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        #overwrite_chars = " " * (num_chars_printed - len(transcript))
        

#         if not result.is_final:
#             pass
#             #sys.stdout.write(transcript + overwrite_chars + "\r")
#             #sys.stdout.flush()

#            # num_chars_printed = len(transcript)

#         else:
#             print(transcript + overwrite_chars)

#             # Exit recognition if any of the transcribed phrases could be
#             # one of our keywords.
#             if re.search(r"\b(exit|quit)\b", transcript, re.I):
#                 print("Exiting..")
#                 break

#             num_chars_printed = 0

        translate_text(text=transcript + overwrite_chars)

            
            
            
def main():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "ja-JP"  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)

if __name__ == "__main__":
    main()

NameError: name 'overwrite_chars' is not defined